# Лабораторна робота 3 з "Асиметричних криптосистем та протоколів"
## Тема: Криптосистема Рабіна. Атака на протокол доведення без розголошення.

**Виконали**\
Бондар Петро, ФІ-03\
Кістаєв Матвій, ФІ-03

In [25]:
import hashlib
import random as rnd
import requests
import sympy
from optimus import *

In [26]:
KEY_LENGTH = 512


class Rabin_Server:
    __base_url = 'http://asymcryptwebservice.appspot.com/rabin/'
    s = requests.Session()
    n = None
    b = None
    
    # Setup server private key and receive server pub key
    def set_server_key(self, key_l: int) -> (str, str):
        req = f'{self.__base_url}serverKey?keySize={key_l}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        self.n, self.b = (int(r['modulus'], 16), int(r['b'], 16))
        return (self.n, self.b)
    
    # Ask server to encrypt
    def encrypt(self, M: str, rec_n, rec_b, type='TEXT'):
        req = f'{self.__base_url}encrypt?modulus={format(rec_n, "X")}&b={format(rec_b, "X")}&message={M}&type={type}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return (r['cipherText'], r['parity'], r['jacobiSymbol'])
    
    # Ask server to decrypt this message with his private keys
    def decrypt(self, C: str, p: int, j: int, type='TEXT'):
        req = f'{self.__base_url}decrypt?cipherText={C}&expectedType={type}&parity={p}&jacobiSymbol={j}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['message']
    
    # Ask server to sign this message with his private keys
    def sign(self, M: str, type='TEXT'):
        req = f'{self.__base_url}sign?message={M}&type={type}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['signature']
    
    # Verify the message using this public key
    def verify(self, M: str, sign: str, u_n, type='TEXT'):
        req = f'{self.__base_url}verify?message={M}&type={type}&signature={sign}&modulus={format(u_n, "X")}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['verified']


In [27]:
def str2hex(s: str):
    res = ""

    for c in s:
        cb = hex(ord(c))
        res += cb[2::]

    return res

def num2str(n: int):
    text = str()
    while n != 0:
        text += chr(n % 256)
        n //= 256
    
    return text[::-1]

def rabin_format(X: int):
    return 255*(2**(KEY_LENGTH*2 - 16)) + X * (2**64) + random.randrange(0, 2**64)

def rabin_unformat(X: int):
    return (X % (pow(2, (KEY_LENGTH*2 - 16)))) // (2**64)

In [28]:
class Rabin_User:
    def __init__(self, p, q, b = 0, serv = Rabin_Server()):
        print("Initializing user...")
        if not check_prime(p) or not check_prime(q):
            raise RuntimeError("p or q is not a prime number.")
        
        self.serv = serv
        self.p = p
        self.q = q
        self.b = b
        self.n = p*q
        self.get_server_public_key(KEY_LENGTH * 2)

        print(f"User private key (p, q): {(self.p, self.q)}")
        print(f"User public key (n, b): {(self.n, self.b)}")
        print(f"User server public key (n, b): {(self.serv.n, self.serv.b)}")
        print("--------------------------------------------------------")
    

    def get_server_public_key(self, len: int):
        self.serv.set_server_key(len)


    def send_message(self, M: str):
        print("Sending message to the server...")
        print(f"Sent message: {M}")

        M_format = rabin_format(int(str2hex(M), 16))
        
        c1 = ((M_format + (self.serv.b * pow(2, -1, self.serv.n))) % self.serv.n) % 2
        c2 = int(sympy.jacobi_symbol(M_format + (self.serv.b * pow(2, -1, self.serv.n)), self.serv.n) == 1)
        C = format(M_format*(M_format + self.serv.b) % self.serv.n, "X")
        print(f"Sent cyphertext: {(C, c1, c2)}")
        

        M1 = self.serv.decrypt(C, c1, c2)
        print(f"Server responce: {M1}")

        check = (M1 == M)
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")


    def send_message_sign(self, M: str):
        print("Sending signature to the server...")
        print(f"Sent message: {M}")
        
        X = rabin_format(int(str2hex(M), 16))
        while not (sympy.jacobi_symbol(X, self.p) == 1 and sympy.jacobi_symbol(X, self.q) == 1):
            X = ((X >> 64) << 64) + random.randrange(0, 2**64)

        Roots = sqrt_modpq(X, self.p, self.q)

        S = format(Roots[random.randrange(0, 4)], "X")
        print(f"Sent signature: {S}")

        check = self.serv.verify(M, S, self.n)
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")


    def receive_message(self, M: str):
        print("Sending request for message to the server...")
        print(f"Sent message: {M}")
        
        C, b1, b2 = self.serv.encrypt(M, self.n, self.b)
        print(f"Received cyphertext: {C}")

        Roots = sqrt_modpq(int(C, 16) + (self.b**2 * pow(2, -2, self.serv.n) % self.serv.n), self.p, self.q)
        for m in Roots:
            if [b1, b2] == [(m % self.n) % 2, (sympy.jacobi_symbol(m, self.n) == 1)]:
                M1 = (m - (self.b * pow(2, -1, self.serv.n))) % self.serv.n
                break
        
        M2 = num2str(rabin_unformat(M1))
        print(f"Decoded cyphertext: {M2}")

        check = (M == M2)
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")
        
        
    def receive_message_sign(self, M: str):
        print("Sending request for message signature to the server...")
        print(f"Sent message: {M}")
        
        S = self.serv.sign(M)
        print(f"Received signature: {S}")
        M1 = num2str(rabin_unformat(pow(int(S, 16), 2, self.serv.n))) 
        print(f"Message signed with signature: {M1}")
        
        check = (M1 == M)
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")


In [29]:
print(f"Довжина модуля сервера: {2*KEY_LENGTH}")
print(f"Ключі для користувача ({KEY_LENGTH} бітів для кожного простого):")
p1 = generate_blum_prime(KEY_LENGTH)
print(f"p1 = {p1}")
q1 = generate_blum_prime(KEY_LENGTH, [p1])
print(f"q1 = {q1}")

Довжина модуля сервера: 1024
Ключі для користувача (512 бітів для кожного простого):
p1 = 9122212710638655747529840359026772005147515295675216961934158597362832559224537721619889610798591608202291210279762777385361025293605644456129235836549259
q1 = 12473564303486936059441310982382641973739497149616172731895682217115662807115714712474694884578485513523937430525724851584456067812357131603895359943443971


In [30]:
u_r = Rabin_User(p1, q1, b=rnd.randrange(0, 2**KEY_LENGTH))
u_r.send_message("sdjdfgjdhgfjdgfhj")
u_r.receive_message("Hello!")
u_r.send_message_sign("Hello!")
u_r.receive_message_sign("Hello!")

Initializing user...
H: Sent request: http://asymcryptwebservice.appspot.com/rabin/serverKey?keySize=1024
S: Response: {'b': '7874B64116D47A03D10BBC89B0702DE6A0BF8CB0946774E98A9FE528D7CCD92525AF5439651D6B8F66F6067A3AC8910FA5815D5E08983B71BA7756F43995EA4C0164B0AF6EBEC8AA4BBB53C4ECC50F9708CEB7BF98EC770E24978B0168A719577576B3187E0AAD51AAB62EEE428D57A31B12E3EA32ADB6F8ED0E925550D1F4F9', 'modulus': '84FD161F0F3982F6719DBE0AF337BA18B8A58BD9C3AC8E76F1791E3E49C32167B9FE838F8F1B117344D7AB24B1116D3A5B89CEAB8CFF1C91673CD28845BC39A6735A7EAC2B546174639E00EFBFC187490D3D8C01171DD9FBDFD1D620794CDAD680C9AAF5C15581C097E1C24D0E62DD6F86C69E6BEECF05A0E93F3E0FAC270D1D'}
User private key (p, q): (9122212710638655747529840359026772005147515295675216961934158597362832559224537721619889610798591608202291210279762777385361025293605644456129235836549259, 12473564303486936059441310982382641973739497149616172731895682217115662807115714712474694884578485513523937430525724851584456067812357131603895359943443971)
User 

In [31]:
class ZNP_Server:
    __base_url = 'http://asymcryptwebservice.appspot.com/znp/'
    s = requests.Session()
    n = None
    
    # Setup server private key and receive server pub key
    def set_server_key(self) -> (str, str):
        req = f'{self.__base_url}serverKey'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        self.n = int(r['modulus'], 16)
        return self.n
    
    def take_sqrt_mod_n(self, y):
        req = f'{self.__base_url}challenge?y={format(y, "X")}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['root']


class ZNP_User:
    def __init__(self, serv = ZNP_Server()):
        print("Initializing user...")
        
        self.serv = serv
        self.get_server_public_key()

        print(f"User server public key (n): {self.serv.n}")
        print("--------------------------------------------------------")

    def get_server_public_key(self):
        self.serv.set_server_key()

    # Ask server to 
    def attack_server(self):
        print("Starting ZNP attack...")

        print(f"Server public key: {self.serv.n}")

        itr = 0
        while True:
            x1 = rnd.randint(1, self.serv.n)
            itr += 1

            if sympy.jacobi_symbol(x1, self.serv.n) != -1:
                continue
            print(f"Randomed {itr} times")

            print("\n==== Asking to take root =====")
            x2 = int(self.serv.take_sqrt_mod_n(pow(x1, 2, self.serv.n)), 16)
            print("========================")
            print(f"\nCandidate: {x2}")
            
            if x1 != x2 and x1 != ((-x2) % self.serv.n):
                print("Candidate is OK!")
                break

        p = math.gcd(x1 - x2, self.serv.n)
        if p == 1:
            p = math.gcd(x1 + x2, self.serv.n)
        q = self.serv.n // p

        print("\n== Results ==")
        print(f"p: {p}")
        print(f"q: {q}")
        print(f"p*q: {p*q}")
        print("===========\n")

        check = (p*q == self.serv.n)
        if check:
            print("Success!")
        else:
            print("No Success. ЩЗХ? :(")
        
        print("--------------------------------------------------------")

In [32]:
znp_u = ZNP_User()
znp_u.attack_server()

Initializing user...
H: Sent request: http://asymcryptwebservice.appspot.com/znp/serverKey
S: Response: {'modulus': 'BD6BAE781A3B59F971C838CDCE57F491FCA63530C3FE869F7C3C2F6D48AF55CF3C1BD459DFDC0CCFF6516BD1BD7CC574E669591E35D9B6097BF8988AF4750ACF38224D00F52FD5764912BCE2FEAC1B8BFA1A9E42F48FE76638DE27381C293568009EA978EBE342321B5B2CF3098F4290B229FCAB8134CAEF33959374042C05A0A7143C6F81D57B379AA2B1F623E13A4F0B9A5BB498ECF4DBCE7EA976DF4F03F00D3BB9AED00B0976A109A569BB90EF39F5A175ECC352B1760BEA4ADF8B896F7FB11F005896B600C87A0A77E4899FD380CA646F07CDCAB941C3C099C234D6B2494930F87E97171C3E00DBB8264CAC8AA6BEF8B0C98FE2EBF0B0D28FC6036AE5F1'}
User server public key (n): 2391213937520966371023602746094647798071409078034039297818445565676129928990773115016649254718711108667545426820747751603872944966925015930060572346992463222483432865477688726792859697370508478153381620177694160888287860341601820270870868756774832761464658593864598445349982964736170741711504861444030773015595948305413012062987778147499642